# Introduction to Gurobi in Python
## By Jamie Fairbrother

## Optimization software

## What is Gurobi

- Gurobi is a commercial solver for mixed integer linear and quadratic optimization problems
- Free academics licences are available and do not have any limitations

## How to use Gurobi?

- Interactive shell
- Interfaces in supported programming languages:
    - C/C++
    - .NET
    - Java
    - Python
    - Matlab
    - R
    - Julia (third-party)

## Why use Gurobi with a programming language?

- Cannot easily change models in LP files
- Models in AMLs cannot be easily incorporated into larger processes/algorithms (e.g. some changepoint algorithms involve solving linear programs)
- Some complex solution algorithms can only be implemented in a programming language

## Why use Gurobi with Python?

- Python is a high-level programming language with many packages for scientific computing
- Python interface does not suffer from same limitations as API for other high-level languages (e.g. R and Matlab)

## An example program

\begin{aligned}
\mathrm{minimize}\ & 3 x + 2 y \\
\text{subject to } & x + y \leq 8\\
& x\geq 0\\
& y\in \{0,1\}
\end{aligned}

In [2]:
from gurobipy import *

m = Model()

x = m.addVar(name="x")
y = m.addVar(vtype='b', name="y")
m.addConstr(x + y <= 8, "c0")
m.setObjective(3*x + 2*y, GRB.MAXIMIZE)

m.optimize()

ImportError: libgurobi90.so: cannot open shared object file: No such file or directory

### Process

1. Initialise `Model` object
2. Construct problem variables
3. Construct problem constraints
4. Set objective
5. Optimize
6. Extract solution

# Initialising Model

- A `Model` contains all the data associated with an optimization model.
- It can be initialised with no arguments or optionally a `name` argument

In [2]:
from gurobipy import Model, GRB

model = Model('Example')
model

<gurobi.Model Continuous instance Example: 0 constrs, 0 vars, No parameter changes>

# Constructing Problem Variables

- Individual variables can constructed with single from a model object using the `addVar` method
- This has the following main optional keyword arguments:
    - `name`: name of variable
    - `lb`: Lower bound for new variable (default 0)
    - `ub`: Upper bound for new variable
    - `vtype`: type of decision.
        Allowed options are 'C' for continuous (default), 'B' for binary, 'I' for integer, 'S' for semi-integer, 'N' for semi-integer
    - `obj`: co-efficient of variable in objective

In [3]:
x = model.addVar(vtype='B') # Add single binary variable
model.update()

In [4]:
y = model.addVar(name='y') # Add single continuous non-negative varible

To create a free variable, you have to set the lower bound explicitly to minus infinity:

In [5]:
z=model.addVar(lb=-GRB.INFINITY)

## Adding groups of variables
- When creating structured models one typically needs to create blocks of variables over one or more indices.
- One can do this by simply by creating variables over a loop and assigning the result to a dictionary (or some other collection):

In [6]:
x = dict()
for i in range(5):
    for j in range(i, 5):
        x[i,j] = model.addVar()

Alternatively, one can use the `addVars` method to add multiple variables with a single function call. This takes the same optional arguments as `addVar`.

In [7]:
x = m.addVars(3, lb=-GRB.INFINITY) # Create block of 3 free variables
y = m.addVars(5, 4, vtype='i') # Create 5x4 block of variables

In [8]:
z = m.addVars((i,j) for i in range(5) for j in range(i,5))

# Adding constraints

- Individual constraints can be added using the `addConstr` method of `Model`
- Operator overloading allows one to build constraints from problem variables in an intuitive way:
- `name` keyword argument can be used to provide a name to the constraint

In [9]:
m = Model()
x = m.addVar()
y = m.addVar()
z = m.addVar()
m.addConstr(x + y <= 1)
m.addConstr(2*z + x == 2, name='Cons2')

<gurobi.Constr *Awaiting Model Update*>

### Warning
- `==` operator not `=` is used to create an equality constraint
- A constraint can only have a single comparison operator, that is, we cannot write expressions such as:
```
2 <= 2*x + 3*y <= 7
```

## Quicksum and using expressions

To create a constraint which sums over a block of variables, we can use the `quicksum` function:

In [10]:
from gurobipy import quicksum

m = Model()
x = m.addVars(10) # Block of 10 non-negative variables
m.addConstr(quicksum(x) == 5.0)

<gurobi.Constr *Awaiting Model Update*>

Expressions can be created outside of constraints for reuse

In [11]:
expr = quicksum(i * x[i] for i in range(10))
m.addConstr(expr <= 20)

<gurobi.Constr *Awaiting Model Update*>

Like variables, groups of constraints can be added using `for` loops, or using the `addConstrs` method of `Model`.
The following code creates the constraints:

\begin{aligned}
x_i &\leq i \qquad \text{for } i = 1,\ldots,10\\
\sum_{i=1}^{10} y_{ij} &= 1 \qquad \text{for } j = 1,\ldots,10
\end{aligned}

In [12]:
y = m.addVars(10, 10) # Block of 10x10 non-negative variables
m.addConstrs(x[i] <= i for i in range(10))
m.addConstrs((quicksum(y[i,j] for i in range(10)) == 1.0 for j in range(10)), name="group_cons")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>}

# Setting the objective and setting model parameters

Setting the objective is done via the `setObjective` method of `Model`. This takes arguments an expression to optimize and the `sense` of the objective. This latter argument can take the values `GRB.MINIMIZE` for a minimization, and `GRB.MAXIMIZE` for a maximization.

In [13]:
m.setObjective(quicksum(x) + quicksum(y), GRB.MAXIMIZE)

The behaviour of the solver can be configured through `Model` parameters. These can be set in two ways:

In [14]:
model.setParam("TimeLimit", 90) # Set TimeLimit for solve to 90 seconds

Changed value of parameter TimeLimit to 90.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [15]:
model.Params.TimeLimit = 90

Parameter TimeLimit unchanged
   Value: 90.0  Min: 0.0  Max: inf  Default: inf


There are many options available. The following are commonly used:
- `MIPGap`: relative optimality gap at which to terminate optimization (default is 1e-4)
- `OutputFlag`: Enables (value 1) or disables (value 0) printing of progress of optimization to screen (default is 1)
- `Threads`: Number of threads to use in optimization (defaults to number of available processors)

A full list of available parameters can be found here: https://www.gurobi.com/documentation/9.0/refman/parameters.html#sec:Parameters

# Optimizing and extracting solution

The model can now be solved using the `optimize` method:

In [18]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 22 rows, 110 columns and 129 nonzeros
Model fingerprint: 0xaa5fb82f
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 20 rows and 101 columns
Presolve time: 0.01s
Presolved: 2 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+01   1.510395e+01   0.000000e+00      0s
       8    1.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds
Optimal objective  1.500000000e+01


Although from the output we can see that the problem solved to optimality in this case, for cases where the solver is not being used interactively, it is important to check the `status` of the model after `optimize` has been called:

In [19]:
m.status

2

In this case `2` means that an optimal solution was found.
This is not the only case to consider however. 
It is possible the solver found the model to be infeasible, or was unable to find an optimal solution for some other reason.
The possible codes can be found at https://www.gurobi.com/documentation/9.0/refman/optimization_status_codes.html.

If the problem solved to optimality, or the solver found a solution, the solution and objective value can be extracted as follows:

In [20]:
m.objVal # optimal objective value

15.0

In [21]:
x_val = m.getAttr('x', x)
x_val

{0: 0.0,
 1: 0.0,
 2: 2.0,
 3: 0.6666666666666666,
 4: 0.0,
 5: 0.0,
 6: 2.3333333333333335,
 7: 0.0,
 8: 0.0,
 9: 0.0}